## Segmenting and Clustering Neighborhoods in Toronto

In [108]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install lxml html5lib beautifulsoup4

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [109]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [110]:
dfs = pd.read_html(url)

print(len(dfs))

3


In [111]:
df = dfs[0]

In [112]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [113]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
toronto_neighborhoods = pd.DataFrame(columns=column_names)

In [114]:
toronto_neighborhoods,df.shape

(Empty DataFrame
 Columns: [Postal Code, Borough, Neighborhood]
 Index: [],
 (180, 3))

In [115]:
for index in range(df.shape[0]):
    if df['Borough'][index] != 'Not assigned':
        postcode = df['Postal Code'][index]
        borough = df['Borough'][index]
        neighbor = df['Neighbourhood'][index]
        if neighbor == 'Not assigned':
            neighbor = df['Borough'][index]
        toronto_neighborhoods = toronto_neighborhoods.append({
            'Postal Code': postcode,
            'Borough': borough,
            'Neighborhood': neighbor}, ignore_index = True)


    #print(df['Postal Code'][index])
#    neighborhoods = neighborhoods.append({'Borough': borough,
#                                          'Neighborhood': neighborhood_name,
#                                          'Latitude': neighborhood_lat,
#                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [116]:
toronto_neighborhoods.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [117]:
toronto_neighborhoods.shape

(103, 3)

## Get the latitude and the longitude coordinates of each neighborhood

In [118]:
!pip install geocoder

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [119]:
coordinate = pd.read_csv('https://cocl.us/Geospatial_data')

In [120]:
coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [121]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood','Latitude','Longitude'] 

# instantiate the dataframe
toronto_neighbor = pd.DataFrame(columns=column_names)
for index in range(toronto_neighborhoods.shape[0]):
    postcode = toronto_neighborhoods['Postal Code'][index]
    borough = toronto_neighborhoods['Borough'][index]
    neighbor = toronto_neighborhoods['Neighborhood'][index]
    data = coordinate[coordinate['Postal Code'] == postcode]
    lat = data['Latitude'].values
    lon = data['Longitude'].values
    toronto_neighbor = toronto_neighbor.append({
            'Postal Code': postcode,
            'Borough': borough,
            'Neighborhood': neighbor,
            'Latitude': lat[0],
            'Longitude': lon[0]}, ignore_index = True)


In [122]:
toronto_neighbor.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [123]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighbor['Borough'].unique()),
        toronto_neighbor.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [124]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [126]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighbor['Latitude'], toronto_neighbor['Longitude'], toronto_neighbor['Borough'], toronto_neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [143]:
df = toronto_neighbor[['Latitude','Longitude']]#[toronto_neighbor['Borough']=='Downtown Toronto'] # data with Borough = Downtown Toronto

In [144]:
df.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494


# Cluster Neighborhoods
### Run k-means to cluster the neighborhood into 3 clusters.

In [145]:
# one hot encoding
# set number of clusters
kclusters = 3


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 2, 0, 2, 1, 0, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [147]:
# add clustering labels
toronto_neighbor.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = toronto_neighbor

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_neighbor.head() # check the last columns!

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,1,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Finally, let's visualize the resulting clusters

In [148]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_neighbor['Latitude'], toronto_neighbor['Longitude'], toronto_neighbor['Neighborhood'], toronto_neighbor['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters